In [1]:
#importing required libraries
import pandas as pd
import json

In [2]:
# Loading Json file from local and storing in brands pandas dataframe after normalisaing
data=[]
with open("C:/Users/saisu/Downloads/Fetch_Rewards/brands.json", "r") as read_file:
    for line in read_file:
        data.append(json.loads(line))
brands=pd.json_normalize(data)

In [3]:
#glimpse of expanded dataframe
brands.head(2)

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS


In [4]:
brands.shape # there are 1167 brands with 1160 unique barcodes

(1167, 9)

In [5]:
#some brands have same barcodes, this hinders the join between receipts and brands, i.e, required other column to join on - 
# "Brandcode". But since Brandcode column has some dulpicates, these needs to be identified and seperated out for join on both columns.
duplicated_brandcodes=brands[brands.duplicated(['barcode'])]['barcode'].tolist()
print("duplicated_brandcodes:") 
duplicated_brandcodes

duplicated_brandcodes:


['511111504139',
 '511111504788',
 '511111204923',
 '511111305125',
 '511111605058',
 '511111704140',
 '511111004790']

In [6]:
# seperating duplicated and non duplicated barcodes into different dataframes
duplicated_brands=brands[brands['barcode'].isin(duplicated_brandcodes)]
non_duplicated_brands=brands[~brands['barcode'].isin(duplicated_brandcodes)]

In [7]:
## Loading Json file from local and storing in users pandas dataframe after normalisaing
data=[]
with open("C:/Users/saisu/Downloads/Fetch_Rewards/users.json", "r") as read_file:
    for line in read_file:
        data.append(json.loads(line))
users=pd.json_normalize(data)
users=users.drop_duplicates() #duplicate users
users=users.rename(columns={'_id.$oid':'userId'})
users.shape # there are 212 unique users 

(212, 7)

In [8]:
#glimse of users dataframe
users.head(2)

,active,role,signUpSource,state,userId,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12


In [9]:
# Loading Json file from local and storing in receipts pandas dataframe after normalisaing
data=[]
with open("C:/Users/saisu/Downloads/Fetch_Rewards/receipts.json", "r") as read_file:
    for line in read_file:
        data.append(json.loads(line))
receipts=pd.json_normalize(data, max_level = 2)

In [10]:
receipts.shape 
# there are 1119 receipts being uploaded, but each receipt can have many list of items " reward receipt item list" - needs to be expanded

(1119, 15)

In [11]:
receipts.head(5)

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


In [12]:
# expanding rewards receipt item list into a dataframe and joining the receipt from which it it derived from. 
# it has to be noted that there are some nulls in receipts item list. which have been captured by the try except
recipts_items=pd.DataFrame()
for i in range(0,receipts.shape[0]):
    try:
        receipts2=pd.json_normalize(receipts["rewardsReceiptItemList"][i])
        receipts2['_id.$oid']=receipts["_id.$oid"][i]
        recipts_items=pd.concat([recipts_items,receipts2])
    except:
        recipts_items_empty=pd.DataFrame()
        recipts_items_empty["rewardsReceiptItemList"]=''
        recipts_items_empty['_id.$oid']=receipts["_id.$oid"][i]
        recipts_items=pd.concat([recipts_items,recipts_items_empty])
recipts_items.shape # there are 6941 items purchases from all the recipts

(6941, 36)

In [13]:
recipts_items.head(5) # glimse of recipts_items dataframe

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,rewardsReceiptItemList,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# merging items full data expanded and someother receipt level information on recipt id and storing in a dataframe
receipts_full_data=pd.merge(recipts_items,receipts,on='_id.$oid',how='outer')
receipts_full_data.shape

(7381, 50)

In [15]:
## Now we have all the receipts expanded to items level, brands and users dataframes in a tabular form 
## we need to merge all the dataframes.

##Join1 - Joining users on receipts( expanded full data) on user_id - uploaded to app  
##Join2 - Joining the resultant new dataframe into brands on barcode and brandcode. 
##        This join need to be made considering the barcode first and if duplicated 
##        barcodes then need to be joined on brandcode. bEcause brandcodes has some nulls. 

In [16]:
receipts_full_data.head(2)

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,rewardsReceiptItemList_y,rewardsReceiptStatus,totalSpent,userId,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12


In [17]:
#Join1
full_data=receipts_full_data.merge(users,on='userId',how='outer')
full_data.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date,active,role,signUpSource,state,createdDate.$date,lastLogin.$date
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,1.609688e+12,1.609688e+12,1.609688e+12,1.609632e+12,True,consumer,Email,WI,1.609688e+12,1.609688e+12
1,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,1.609688e+12,1.609688e+12,1.609688e+12,1.609632e+12,True,consumer,Email,WI,1.609688e+12,1.609688e+12
2,4011,ITEM NOT FOUND,20.00,20.00,False,1,True,5.0,4011,True,...,1.609688e+12,1.609688e+12,1.609688e+12,1.609632e+12,True,consumer,Email,WI,1.609688e+12,1.609688e+12
3,4011,ITEM NOT FOUND,20.00,20.00,False,1,True,3.0,4011,True,...,1.609688e+12,1.609688e+12,1.609688e+12,1.609632e+12,True,consumer,Email,WI,1.609688e+12,1.609688e+12
4,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,2.0,4011,True,...,1.609688e+12,1.609688e+12,1.609688e+12,1.609632e+12,True,consumer,Email,WI,1.609688e+12,1.609688e+12


In [18]:
full_data.shape

(7452, 56)

In [19]:
brands.head(2)

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS


In [20]:
#Join user2 - non duplicated brands joined on barcode
# full_data_2=full_data.merge(non_duplicated_brands,left_on='barcode',right_on='barcode',how='outer')
# full_data_2.shape

In [22]:
# removing duplicated data
full_data_without_duplicated_brands=full_data[~full_data['barcode'].isin(duplicated_brandcodes)]
# duplicated data
full_data_with_duplicated_brands=full_data[full_data['barcode'].isin(duplicated_brandcodes)]

In [ ]:
full_data_without_duplicated_brands.shape

In [26]:
full_data1=full_data_without_duplicated_brands.merge(non_duplicated_brands,on=['barcode'],how='outer')

In [27]:
# duplicated data should be joined on barcode and brandcode #non_duplicated_brands
full_data2=full_data_with_duplicated_brands.merge(duplicated_brands,on=['barcode','brandCode'],how='outer')

In [28]:
full_data1.shape

(8583, 64)

In [31]:
full_data2.head(20)

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,state,createdDate.$date,lastLogin.$date,category,categoryCode,name,topBrand,_id.$oid_y,cpg.$id.$oid,cpg.$ref
0,511111704140,PREGO,2.19,2.19,NaN,1307,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
1,511111704140,PREGO,2.19,2.19,NaN,1369,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
2,511111704140,PREGO,2.19,2.19,NaN,1361,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
3,511111704140,PREGO,2.69,2.69,NaN,1774,NaN,1.0,NaN,NaN,...,WI,1.611609e+12,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
4,511111704140,PREGO,2.69,2.69,NaN,1776,NaN,1.0,NaN,NaN,...,WI,1.611609e+12,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
5,511111704140,PREGO,2.69,2.69,False,1956,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
6,511111704140,PREGO,2.69,2.69,False,1958,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
7,511111504788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Baking,NaN,test,NaN,5c408e8bcd244a1fdb47aee7,59ba6f1ce4b092b29c167346,Cogs
8,511111305125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Baby,NaN,Chris Image Test,NaN,5c4699f387ff3577e203ea29,55b62995e4b0d8e685c14213,Cogs
9,511111504139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Beverages,NaN,Chris Brand XYZ,NaN,5a7e0604e4b0aedb3b84afd3,55b62995e4b0d8e685c14213,Cogs


In [32]:
#merge duplicated and non duplicated data
final_full_data=pd.concat([full_data1,full_data2],axis=1)
final_full_data.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,state,createdDate.$date,lastLogin.$date,category,categoryCode,name,topBrand,_id.$oid_y,cpg.$id.$oid,cpg.$ref
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
1,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
2,4011,ITEM NOT FOUND,20.00,20.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
3,4011,ITEM NOT FOUND,20.00,20.00,False,1,True,3.0,4011,True,...,WI,1.611609e+12,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs
4,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,2.0,4011,True,...,WI,1.611609e+12,NaN,Condiments & Sauces,NaN,Prego,False,5a8c344ae4b0ccf165fac9e5,5a734034e4b0d58f376be874,Cogs


In [33]:
#for staging table
final_full_data.to_excel("C:/Users/saisu/Downloads/Fetch_Rewards/final_full_data.xlsx")